To run this code:

* Download nasbench code and place in external
* Download nasbench data and place in external/nasbench

In [90]:
import sys
sys.path.append("external/nasbench101")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

import torch

from nasbench import api

nasbench = api.NASBench('./external/nasbench101/nasbench_only108.tfrecord')
print("Done loading dataset")

INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'

Loading dataset from file... This may take a few minutes...


Loaded dataset in 46 seconds
Done loading dataset


Define a bunch of test models 

In [91]:
model_spec_demo = api.ModelSpec(
    # Adjacency matrix of the module
    matrix=[[0, 1, 1, 1, 0, 1, 0],    # input layer
            [0, 0, 0, 0, 0, 0, 1],    # 1x1 conv
            [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv
            [0, 0, 0, 0, 1, 0, 0],    # 5x5 conv (replaced by two 3x3's)
            [0, 0, 0, 0, 0, 0, 1],    # 5x5 conv (replaced by two 3x3's)
            [0, 0, 0, 0, 0, 0, 1],    # 3x3 max-pool
            [0, 0, 0, 0, 0, 0, 0]],   # output layer
    # Operations at the vertices of the module, matches order of matrix
    ops=[INPUT, CONV1X1, CONV3X3, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT])


model_spec_full = api.ModelSpec(
    # Adjacency matrix of the module
    matrix=[[0, 1, 1, 1, 1, 1, 1],    
            [0, 0, 1, 1, 1, 1, 1],    
            [0, 0, 0, 1, 1, 1, 1],    
            [0, 0, 0, 0, 1, 1, 1],    
            [0, 0, 0, 0, 0, 1, 1],    
            [0, 0, 0, 0, 0, 0, 1],    
            [0, 0, 0, 0, 0, 0, 0]],   
    # Operations at the vertices of the module, matches order of matrix
    ops=[INPUT, CONV1X1, CONV3X3, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT])

model_spec_full_small = api.ModelSpec(
    # Adjacency matrix of the module
    matrix=[[0, 1, 1, 1],    # input layer
            [0, 0, 1, 1],
            [0, 0, 0, 1],
            [0, 0, 0, 0]],   # output layer
    # Operations at the vertices of the module, matches order of matrix
    ops=[INPUT, CONV3X3, CONV3X3, OUTPUT])

model_spec_linear = api.ModelSpec(
    # Adjacency matrix of the module
    matrix=[[0, 1, 0, 0, 0, 0, 0],    
            [0, 0, 1, 0, 0, 0, 0],    
            [0, 0, 0, 1, 0, 0, 0],    
            [0, 0, 0, 0, 1, 0, 0],    
            [0, 0, 0, 0, 0, 1, 0],    
            [0, 0, 0, 0, 0, 0, 1],    
            [0, 0, 0, 0, 0, 0, 0]],   
    # Operations at the vertices of the module, matches order of matrix
    ops=[INPUT, CONV1X1, CONV1X1, CONV1X1, CONV1X1, CONV1X1, OUTPUT])

model_spec_linear_small = api.ModelSpec(
    # Adjacency matrix of the module
    matrix=[[0, 1, 0],    # input layer
            [0, 0, 1],
            [0, 0, 0]],   # output layer
    # Operations at the vertices of the module, matches order of matrix
    ops=[INPUT, CONV3X3, OUTPUT])


model_spec_test = api.ModelSpec(
    # Adjacency matrix of the module
    matrix=[[0, 1, 1, 1, 0, 1, 0],    
            [0, 0, 0, 0, 0, 0, 1],    
            [0, 0, 0, 0, 0, 0, 1],    
            [0, 0, 0, 0, 1, 0, 0],    
            [0, 0, 0, 0, 0, 0, 1],    
            [0, 0, 0, 0, 0, 0, 1],    
            [0, 0, 0, 0, 0, 0, 0]],   
    # Operations at the vertices of the module, matches order of matrix
    ops=[INPUT, CONV1X1, CONV3X3, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT])


Build Pytorch and Tensorflow models, copy parameters, and run model on a single example

In [92]:
from nasbench.lib import config as _config

from util import nasbench_tensorflow_model_builder, copy_tensorflow_params_to_pytorch
from model import NasBenchNet

import tensorflow as tf
import numpy as np

is_training = True
config = _config.build_config()


def compare_tf_and_pytorch(model_spec, batch_size=16):
    tf.reset_default_graph()

    with tf.Session() as sess:
        # Build  models
        model_spec = model_spec_demo
        in_shape = (None, 32, 32, 3)

        infeat, net = nasbench_tensorflow_model_builder(
            model_spec, config, in_shape, is_training
        )
        pytorch_model = NasBenchNet(3, model_spec, config).float()

        ## Uncomment to visualize graph in Tensorboard
        # writer = tf.summary.FileWriter("/tmp/log/", sess.graph)
        # writer.close()

        # Run tensorflow initialization
        sess.run(tf.global_variables_initializer())

        # Copy parameters to pytorch model
        pytorch_model = copy_tensorflow_params_to_pytorch(pytorch_model, sess)

        # Sanity check
        x = np.float32(np.random.randn(*(BATCH_SIZE, *in_shape[1:])))
        result_tf = sess.run(net, feed_dict={infeat: x})

        result_pytorch = pytorch_model(torch.from_numpy(x).permute(0, 3, 2, 1))
        result_pytorch = result_pytorch.cpu().detach().numpy()

        # Check of results match
        return np.mean(np.abs(result_pytorch - result_tf))

err = compare_tf_and_pytorch(model_spec_demo)
print("Average distance between TF and Pytorch result: ", err)


Average distance between TF and Pytorch result:  2.1123933e-05


In [ ]:
import tensorflow as tf
import time

for i, h in enumerate(nasbench.hash_iterator()):
  fixed, _ = nasbench.get_metrics_from_hash(h)
  spec = api.ModelSpec(
      matrix=fixed["module_adjacency"],
      ops=fixed["module_operations"])

  start = time.time()
  err = compare_tf_and_pytorch(spec, 2)
  print("testing hash", i, h, err, time.time()-start)    

  if err > 1e-3:
    print("error too large")

#  if i > 10:
#    break  

print("DONE WITH ALL TESTS")

testing hash 0 00005c142e6f48ac74fdcf73e3439874 1.6901711e-05 5.3442957401275635


testing hash 1 0000718c96730a92ea6d5242f8ce77e2 1.8889457e-05 4.724383354187012


testing hash 2 0000cb5372fad9d62c470df699ac6d52 1.7947843e-05 4.593315124511719


testing hash 3 0000fa05697179112aaf69c2f0a51a0f 1.6611488e-05 5.45121431350708


testing hash 4 00010e19334eb9dcbe1fe61a98d524c2 1.7406954e-05 4.660356283187866


...

testing hash 6663 04043104b0678d89176614148b653801 2.0549294e-05 4.6455535888671875


testing hash 6664 0404382a6b008f0313713a0adf913655 1.7716502e-05 4.766428709030151


testing hash 6665 0404638fc34b45c34a821885bb7a07b9 1.846822e-05 5.558339595794678


testing hash 6666 0404aa144b66f05ca548d9a4f810df26 1.6747555e-05 4.676079273223877


testing hash 6667 0404b34c62d4082337b14eeae8d6f43c 1.6880245e-05 4.665009260177612
